In [2]:
# 1. Enable Internet in the Kernel (Settings side pane)

# 2. Curl cache may need purged if v0.1.6 cannot be found (uncomment if needed). 
# !curl -X PURGE https://pypi.org/simple/kaggle-environments

# ConnectX environment was defined in v0.1.6
!pip install 'kaggle-environments>=0.1.6'

from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render()

     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.4 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
Loading environment football failed: No module named 'gfootball'


**Import Statements**

In [3]:
from numpy import reshape, argmax, add, copy
from random import choice



**Defining Adjacent Blocks**

In [4]:
def up(index):
  return (index[0]-1,index[1])

def down(index):
  return (index[0]+1,index[1])

def left(index):
  return (index[0],index[1]-1)

def right(index):
  return (index[0],index[1]+1)

def up_left(index):
  return (up(left(index)))
  # return (index[0]-1,index[1]-1)

def down_left(index):
  return (down(left(index)))
  # return (index[0]+1,index[1]-1)

def up_right(index):
  return (up(right(index)))
  # return (index[0]-1,index[1]+1)

def down_right(index):
  return (down(right(index)))
  # return (index[0]+1,index[1]+1)

def possible_moves(new_board):
  possibilities = list(-1 for i in range(7))
  for i in range(7):
    for j in range(6):
      if new_board[j][i] == 0:
        possibilities[i] = j
  possible_moves = [(possibilities[i],i) for i in range(7) if possibilities[i] != -1]
  return possible_moves

def next_state(new_board, move, player):
  new_board[move] = player
  return new_board



**Scores for *3 Convertibles* and *2 in a rows***

In [5]:
def find_scores(board, scores, player, score_chart):
  horizontal_choices = list(0 for i in range(7))
  vertical_choices = list(0 for i in range(7))
  diagonal_choices = list(0 for i in range(7))
  possibile_moves = possible_moves(board)
  # print(moves)
  for index in possibile_moves:
    # print(index)
    # 3 Horizontal
    # .11* or .1*1 or .*11 
    if index[1] < 4 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count >= 0 and board[right(right(right(index)))] == player:
        count += 1
      elif board[right(right(right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(left(index)))] == player:
        count += 1
      elif board[left(left(left(index)))] != 0:
        count = -1
      if count >= 0 and board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    
    # 3 Vertical
    if index[0] > 0 and index[0] < 4 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player and board[down(down(index))] == player:
        scores[index[1]] += score_chart[4]
        vertical_choices[index[1]] += 1
    
    # 3 Positive Diagonal
    # .11* or .1*1 or .*11 
    if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(up_right(index)))] == player:
        count += 1
      elif board[up_right(up_right(up_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(down_left(index)))] == player:
        count += 1
      elif board[down_left(down_left(down_left(index)))] != 0:
        count = -1
      if count >= 0 and board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 3 Negative Diagonal
    # .11* or .1*1 or .*11 
    if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(down_right(index)))] == player:
        count += 1
      elif board[down_right(down_right(down_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(up_left(index)))] == player:
        count += 1
      elif board[up_left(up_left(up_left(index)))] != 0:
        count = -1
      if count >= 0 and board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 2 Horizontal
    if horizontal_choices[index[1]] == 0:
      # 2 Horizontal .1**
      if index[1] < 4 and board[right(index)] == player:
        if (board[right(right(index))] == 0 or board[right(right(index))] == player) and (board[right(right(right(index)))] == 0 or board[right(right(right(index)))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *.1*
      if index[1] < 5 and index[1] > 0 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      #2 Horizontal 1.**
      if index[1] < 4 and index[1] > 0 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **.1
      if index[1] < 6 and index[1] > 1 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *1.*
      if index[1] < 6 and index[1] > 1 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **1.
      if index[1] > 2 and board[left(index)] == player:
        if (board[left(left(index))] == 0 or board[left(left(index))] == player) and (board[left(left(left(index)))] == 0 or board[left(left(left(index)))] == player):
          scores[index[1]] += score_chart[0]
      
    # 2 Vertical
    if index[0] > 1 and index[0] < 5 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player:
        scores[index[1]] += score_chart[1]
    
    # 2 Diagonal
    if index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-left-low
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-right-mid
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-right-mid
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-left-high
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
    
    if index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-right-low
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-left-mid
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-left-mid
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-right-high
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      
      # 2 Diagonal up-left-low
      if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
        if board[up_left(index)] == player:
          if (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0) and (board[up_left(up_left(up_left(index)))] == player or board[up_left(up_left(up_left(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal up-right-low
      if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
          if board[up_right(index)] == player:
            if (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0) and (board[up_right(up_right(up_right(index)))] == player or board[up_right(up_right(up_right(index)))] == 0):
              scores[index[1]] += score_chart[2]
      # 2 Diagonal down-right-high
      if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
        if board[down_right(index)] == player:
          if (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0) and (board[down_right(down_right(down_right(index)))] == player or board[down_right(down_right(down_right(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal down-left-low
      if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
          if board[down_left(index)] == player:
            if (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0) and (board[down_left(down_left(down_left(index)))] == player or board[down_left(down_left(down_left(index)))] == 0):
              scores[index[1]] += score_chart[2]

  return scores



**Win Check**

In [6]:
def win_check(board, player):
  possibilities = list(-1 for n in range(7))
  possibile_moves = possible_moves(board)
  for index in possibile_moves:
    # print(index)
    # Horizontal .111
    if index[1] < 4:
      if board[right(index)] == player and board[right(right(index))] == player and board[right(right(right(index)))] == player:
        return index[1]
    # 2 Horizontal 1.11
    if index[1] > 0 and index[1] < 5:
      if board[(left(index))] == player and board[right(index)] == player and board[right(right(index))] == player:
        return index[1]
    # 2 Horizontal 11.1
    if index[1] > 1 and index[1] < 6:
      if board[left(left(index))] == player and board[(left(index))] == player and board[right(index)] == player:
        return index[1]
    # 2 Horizontal 111.
    if index[1] > 2:
      if board[left(left(left(index)))] == player and board[left(left(index))] == player and board[left(index)] == player:
        return index[1]
    
    # Vertical
    if index[0] < 3:
      if board[down(index)] == player and board[down(down(index))] == player and board[down(down(down(index)))] == player:
        return index[1]
    
    # Positive Diagonal
    if index[0] > 2 and index[1] < 4:
      if board[up_right(index)] == player and board[up_right(up_right(index))] == player and board[up_right(up_right(up_right(index)))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5:
      if board[down_left(index)] == player and board[up_right(index)] == player and board[up_right(up_right(index))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6:
      if board[down_left(down_left(index))] == player and board[down_left(index)] == player and board[up_right(index)] == player:
        return index[1]
    if index[0] < 3 and index[1] > 2:
      if board[down_left(down_left(down_left(index)))] == player and board[down_left(down_left(index))] == player and board[down_left(index)] == player:
        return index[1]
    
    # Negative Diagonal
    if index[0] < 3 and index[1] < 4:
      if board[down_right(index)] == player and board[down_right(down_right(index))] == player and board[down_right(down_right(down_right(index)))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5:
      if board[up_left(index)] == player and board[down_right(index)] == player and board[down_right(down_right(index))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6:
      if board[up_left(up_left(index))] == player and board[up_left(index)] == player and board[down_right(index)] == player:
        return index[1]
    if index[0] > 2 and index[1] > 2:
      if board[up_left(up_left(up_left(index)))] == player and board[up_left(up_left(index))] == player and board[up_left(index)] == player:
        return index[1]
  
  return -1



**Define Scorer**

In [7]:
def scorer(board, player, opponent, turn, scores_player, scores_opponent):
    # print(observation.board)
    # print("\nStart of Turn " + str(turn))
    scores = list(0 for j in range(7))
    # print("Current Board:")
    # print(board)
    moves = possible_moves(copy(board))
    # print("All Moves = " + str(moves))
    
    # Only Move
    if len(moves) == 1:
      # print("Only move")
      return moves[0][1], "only"

    # Offensive and Defensive
    our_choice = win_check(copy(board), player)
    if our_choice > -1:
      # print("Seizing at " + str(our_choice))
      return our_choice, "seize"
    else:
      our_choice = win_check(copy(board), opponent)
      if our_choice > -1:
        # print("Blocking at " + str(our_choice))
        return our_choice, "block"
    
    # Preventing Setup
    temp = moves[:]
    for i in temp:
      if len(moves) > 1 and win_check(copy(next_state(copy(board), i, player)), opponent) > -1:
        # print("Removed " + str(i))
        moves.remove(i)
    # print(moves)
    # Last resort
    if len(moves) == 1:
      # print("Last Resort")
      return moves[0][1], "last"
    
    # board = reshape(observation.board,(6,7))
    # print("Current Board: ")
    # print(board)
    # Scoring Moves : 2-step Look ahead
    value_opponent = list(list(-10 for i in range(7)) for i in range(7))
    # print("All moves = " + str(moves))
    for index in moves:
      # print("Current Index = " + str(index))
      next_board = copy(next_state(copy(board), index, player))
      # print(next_board)
      if win_check(copy(next_board), opponent) > -1:
        value_opponent[index[1]] = list(-10 for i in range(7))
      else:
        next_moves = possible_moves(copy(next_board))
        # print("Next Moves = " + str(next_moves))
        value_opponent[index[1]] = find_scores(copy(next_board), list(0 for j in range(7)), opponent, scores_opponent)
      # best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    # print(value_opponent)
    final_value = list(sum(value_opponent[i]) for i in range(7))
    best_moves = [i for i in range(len(final_value)) if final_value[i] == max(final_value)]
    # print("Final Values = " + str(final_value))
    # print("Best Moves (no Priority) = " + str(best_moves))
    return best_moves, "steps"



**Define Agent**

In [33]:
def my_agent(observation, configuration):
    scores_player = [2,2,2,3,3,3,4,4,4,6,6] # 11 possibilities
    scores_opponent = [-2,-2,-2,-4,-4,-4,-4,-4,-4,-10,-10] # 11 possibilities
    board = reshape(observation.board,(6,7))
    turn = observation.step + 2
    player = observation.mark
    opponent = player % 2 + 1
    best_moves, category = scorer(copy(board), player, opponent, turn, scores_player, scores_opponent)
    # print(best_moves)
    # if str(type(best_moves))[8:-2] == 'int':
    if category != "steps":
      return best_moves
    else:  
      our_choice = choice(best_moves)
      scores = [0 for i in range(7)]
      for i in range(7):
        if i in best_moves:
          for x in range(6):
            if board[x][i] == 0:
              move = (x,i)
          next_best_moves, category = scorer(copy(next_state(copy(board), move, opponent)), opponent, player, turn + 1, scores_opponent, scores_player)
          if category == "seize":
            scores[i] += 10
          elif category == "block":
            scores[i] -= 5
          elif category == "steps":
            scores[i] += 1
        else:
          scores[i] = -10
    best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    if len(best_moves) == 0:
      return best_moves[0]
    else:
      scores = add(scores, [0,1,2,3,2,1,0])
      best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
      return best_moves[0]



**Play Using Created Agent**

In [38]:
env.reset()

# Play as the First Player against "random" agent.
# env.run([my_agent, "random"])

# Play as the First Player against "negamax" agent.
# env.run([my_agent, "negamax"])

# Play as the Second Player against "negamax" agent.
env.run(["negamax", my_agent])

# Render graphics of the Game
env.render(mode="ipython", width=500, height=450)

# render just the Game
# env.render()